Define a function that is able to request the inputs manually but also create a dummy dataset from scratch

In [433]:
import random
from datetime import datetime, timedelta
import numpy as np

#set the random seed for reproducibility
random.seed(34)

#define the function
def input_expenses(demo=True):
    '''
    Function to input the expenses. \n 
    Expenses can be added manually or they can be created as a synthetic dataset (demo=True) \n
    Returns a list of dicts being each one an expense.
    '''
    
    #if interactive is true, request to introcute the inputs manually
    if (demo==False):
        exp_date = input("Enter the date of the expense in the format YYYY-MM-DD")
        exp_category = input("Enter the category of the expense")
        exp_amount = input("Enter the amount of the expense")
        exp_description = input("Enter the description of the expense")

        #create a list with a dictionary including the inputs
        list_expenses = [{ \
            "date": exp_date, \
            "category": exp_category, \
            "amount": exp_amount, \
            "description": exp_description \
        }]
    #create a dummy list of inputs from scratch
    else:
        #create random dates within 2024
        #first the days within the range 
        start_date = datetime(2024, 1, 1)
        end_date = datetime(2024, 12, 31)
        delta = end_date - start_date
        #create 20 random dates from the start and convert to the required format
        example_dates = [ \
            (start_date + timedelta(days=random.randint(0, delta.days))).strftime("%Y-%m-%d") for _ in range(100) \
        ]

        #create random expenses categories including missing data
        example_categories = [ \
            random.choice(["Groceries", "Clothing", "Entertaiment", np.nan]) for _ in range(100) \
        ]

        #create random amounts based on the categories
        example_amounts = []
        for i in example_categories:
            if(i=="Groceries"):
                example_amounts.append(random.randint(5,200))
            elif(i=="Clothing"):
                example_amounts.append(random.randint(5,100))
            elif(i=="Entertaiment"):
                example_amounts.append(random.randint(5,20))
            else:
                example_amounts.append(np.nan)

        #create random descriptions based on the categories
        example_descriptions = []
        for i in example_categories:
            if(i=="Groceries"):
                example_descriptions.append( \
                    "doing groceries in " + random.choice(["Aldi", "Lidl", "Tesco", "Dunnes"]) \
                )
            elif(i=="Clothing"):
                example_descriptions.append( \
                    "buying clothes in " + random.choice(["Penneys", "H&M", "Zara", "Dunnes"]) \
                )
            elif(i=="Entertaiment"):
                example_descriptions.append( \
                    "going to the cinema to watch " + random.choice( \
                        ["The Avengers", "Star Wars", "The Lord of the Rings"] \
                    ) \
                )
            else:
                example_descriptions.append(np.nan)

        #create a list of dictionaries with the inputs
        list_expenses = [ \
            { \
                "date": date, \
                "category": category, \
                "amount_usd": amount_usd, \
                "description": description \
            } for date, category, amount_usd, description in zip(example_dates, example_categories, example_amounts, example_descriptions) \
        ]
    
    #return only the dict
    return list_expenses

Define a function to visualize expenses avoiding incomplete entries

In [434]:
import pandas as pd
#list_expenses=input_expenses(True)
def visual_exp(list_expenses):
    '''
    Function to visualize the expenses. \n
    This function expects as input a list of dicts being each dict a expense. \n
    It returns a pandas DF without missing data. One row per expense.
    '''

    #convert the list to a pandas DF
    pd_expenses = pd.DataFrame(list_expenses)

    #remove missing 
    pd_expenses_no_nan = pd_expenses.dropna()

    #convert date column to datatime format for filtering in future steps
    pd_expenses_no_nan.loc[:, "date"] = pd.to_datetime(pd_expenses_no_nan["date"])

    #sort by date
    pd_expenses_no_nan_sorted = pd_expenses_no_nan.sort_values(by='date')
    return pd_expenses_no_nan_sorted

Define function to set and track a monthly budget

In [435]:
#pd_expenses=visual_exp(list_expenses = input_expenses(True))
#monthly_budget=1000
#month=4; year=2024
def budget_tracking(pd_expenses, month, year, monthly_budget):
    '''
    Function to calculate the expenditure in a given month and compare it with the monthly budget. \n
    It takes as input a pandas DF with the expenditure (one row per expense), the month, year and budget. \n
    The function will check whether the user is under or above the monthly limit.
    '''

    #convert the date column to datetime format for filtering
    pd_expenses["date"] = pd.to_datetime(pd_expenses["date"])

    #calculate the current expenditure
    current_expending = pd_expenses.loc[ \
        (pd_expenses["date"].dt.month == month) & (pd_expenses["date"].dt.year == year), \
        "amount_usd" \
    ].sum()
        #select those expenses of the selected month and year
        #extract the amount in USD and sum all of them

    #calculate the difference between budget and current expenditure
    remaining_budget = monthly_budget - current_expending

    #check the user is above or below
    if(remaining_budget < 0):
        print(f"WARNING: You have already exceeded you budget in {remaining_budget} USD")
    else:
        print(f"You have {remaining_budget} USD remaining")


Define function to save expenses in CSV

In [436]:
#pd_expenses=visual_exp(list_expenses = input_expenses(True))
#user_name="dsalazar"
import os
def save_expenses(pd_expenses, user_name):
    '''
    Function to save expenses taking as arguments a pandas DF with the expenses and the user name. \n
    It saves the expenses as a CSV file
    '''

    #create a folder to save the results
    os.makedirs("../expenses/", exist_ok=True)
        #not raise error if the folder exists

    #save the expenses as a CSV file
    pd_expenses.to_csv(\
        "../expenses/expenses_" + user_name + ".csv", \
        sep=",", \
        index=False \
    )

Define function to load previously generated expenses data

In [437]:
#pd_expenses=visual_exp(list_expenses = input_expenses(True))
#user_name="dsalazar"
def load_expenses(user_name):
    
    #define the file path
    file_path = "../expenses/expenses_" + user_name + ".csv"

    #check if the file exists
    if os.path.exists(file_path):
        
        #load it
        loaded_expenses = pd.read_csv(\
            file_path, \
            sep=",", \
            header=0 \
        )
        return loaded_expenses
    else:
        raise ValueError( \
            "ERROR! The user '" + user_name + "' has not previously generated an expenses file" \
        )

Build a function to display a menu with all the previous functionalities

In [438]:


list_expenses = input_expenses(demo=True)
pd_expenses = visual_exp(list_expenses=list_expenses)
budget_tracking(pd_expenses=pd_expenses, month=4, year=2024, monthly_budget=100)
save_expenses(pd_expenses, user_name="dsalazar")
load_expenses(user_name="dsalazar")

You have 36.0 USD remaining


,date,category,amount_usd,description
0,2024-01-01,Clothing,52.0,buying clothes in H&M
1,2024-01-15,Clothing,53.0,buying clothes in Penneys
2,2024-01-16,Entertaiment,10.0,going to the cinema to watch The Avengers
3,2024-01-26,Clothing,27.0,buying clothes in H&M
4,2024-01-31,Clothing,39.0,buying clothes in Zara
...,...,...,...,...
65,2024-10-26,Entertaiment,5.0,going to the cinema to watch Star Wars
66,2024-11-08,Entertaiment,14.0,going to the cinema to watch The Avengers
67,2024-11-08,Groceries,198.0,doing groceries in Dunnes
68,2024-12-09,Clothing,66.0,buying clothes in Dunnes
